In [ ]:
# Run just to test if ollama is running properly

from llama_index.llms.ollama import Ollama

llm = Ollama(model="mistral:latest", request_timeout=1000)

response = llm.complete("What is the history of LEGO?")

print(response)

In [1]:
from pathlib import Path

import qdrant_client
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader
)
from llama_index.llms.ollama import Ollama
from llama_index.core.storage.storage_context import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core.settings import Settings

In [2]:
# Path to your data folder
documents = SimpleDirectoryReader("../data2").load_data()

In [3]:
# Initialize the vector store
client = qdrant_client.QdrantClient(
    path="../qdrant_data"
)
vector_store = QdrantVectorStore(client=client, collection_name="data2")
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [4]:
# Settings for llm (LlamaIndex 0.10)
Settings.llm = Ollama(model="mixtral:latest", embed_model="local", request_timeout=2000, gpu_memory_utilization=0.5)
Settings.embed_model = "local"

In [5]:
# Indexing
index = VectorStoreIndex.from_documents(documents)

# If this throws an OutOfMemoryError, adjust gpu_memory_utilization in settings

In [6]:
# Querying
query_engine = index.as_query_engine()

In [7]:
# Sending the query
response = query_engine.query("The dataset consists of responses to a satisfaction survey. Provide a summary of satisfaction by person assisted by each 'socio implementador u Organización ganadora de Grant Agreement'. Keep it to 50 words per organization.")

In [8]:
# Printing the response
print(response)

 ASOVENTRU (Asociacion de Venezolanos en Trujillo) received high satisfaction ratings, with most respondents reporting excellent assistance and significant improvements in their situations. Some suggested longer support duration for children and more information about service locations.

Accion Contra el Hambre also had positive feedback, with beneficiaries citing good to excellent services, significant improvements, and helpful emprendimiento courses. However, some requested more personalized follow-ups, quicker aid disbursement, and longer course durations.

Encuentros - Servicio Jesuita de la Solidaridad had one response with moderate improvement reported and a suggestion for better follow-up on processes.
